## Необходимые библиотеки

In [1]:
%config InlineBackend.figure_format = 'svg' 
%matplotlib inline 
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
from skimage.feature import canny
from skimage.measure import label, find_contours, regionprops
from skimage.morphology import disk
from skimage.morphology import binary_erosion as erosion
from skimage.morphology import binary_closing as closing
from skimage.morphology import binary_dilation as dilation
from scipy.ndimage import binary_fill_holes as fill_holes
import glob

## Вспомогательные функции, структуры и операции:

In [27]:
pictures = [] # список из изображений
for file in glob.glob("./Samples/*.jpg"):
    pictures.append(str(file))

In [7]:
class Object:
    """
    Структура, отвечающая за отдельно взятую фигуру на изображении
    """
    def __init__(self, figure_matrix, centre):
        self.approx = []
        self.amount_vertexes = 0
        self.figure_matrix = figure_matrix
        self.centre = (round(centre[0]), round(centre[1]))
        self.smoothness = False
        self.convex = False

    def set_approx(self, approx):
        self.approx = np.squeeze(approx, axis=1)
        self.amount_vertexes = approx.shape[0]

    def set_smoothness(self):
        self.smoothness = True

    def del_smoothness(self):
        self.smoothness = False

    def set_convex(self):
        self.convex = True

    def get_convex(self):
        """
        Получить значение выпуклости фигуры (True/False)
        """
        return self.convex 

    def get_smoothness(self):
        """
        Получить значение гладкости фигуры (True/False)
        """
        return self.smoothness

    def get_centre(self):
        """
        Получить координаты центра
        """
        return self.centre

    def get_figure_matrix(self):
        """
        Получить маску фигуры
        """
        return self.figure_matrix

    def get_approx(self):
        """
        Получить точки приближения
        """
        return self.approx

    def get_amount_vertexes(self):
        """
        Получить количество вершин
        """
        return self.amount_vertexes

In [8]:
class Image:
    """
    Структура, отвечающая за все фигуры в одном изображении
    """
    def __init__(self, objects):
        self.objects = objects
        self.amount = len(objects)

    def get_objects(self):
        """
        Возвращает все фигуры класса Object из изображения
        """
        return self.objects

    def get_amount(self):
        """
        Возвращает количество фигур на изображении
        """
        return self.amount

In [9]:
def color_change(image, mode):
    """
    Функция для смены цветов изображения
    """
    if mode == 'grey':
        new_img = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    elif mode == 'rgb':
        new_img = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    elif mode == 'hsv':
        new_img = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    else:
        raise Exception(f'No such mode = {mode}')
    return new_img

In [10]:
def find_the_cards(image):
    """
    Функция для нахождения карт на изображении
    """
    img = color_change(image, 'grey')
    cards = canny(img, sigma=1)  
    cards = dilation(cards, disk(1))
    cards = closing(cards, disk(5))
    cards = fill_holes(cards, disk(1)) 
    cards = erosion(cards, disk(5))
    return cards

In [11]:
def put_mask(image, mask):
    """
    Функция для накладывания маски на изображение
    """
    if len(image.shape) == 2: # если изображение одноканальное
        image[mask == False] = 0
    elif len(image.shape) == 3: # трехканальное
        image[mask == False,:] = 0
    else:
        raise Exception('Incorrect image')
    return image

In [12]:
def angle_between(vec1, vec2):
    """ 
    Угол между двумя векторами (в градусах)
    """
    vec1_unit = vec1 / np.linalg.norm(vec1)
    vec2_unit = vec2 / np.linalg.norm(vec2)
    return np.arccos(np.clip(np.dot(vec1_unit, vec2_unit), -1, 1)) * 57.29

## Вывод картинок

In [34]:
for picture in pictures:
    img = cv.imread(picture)
    img = color_change(img, mode='rgb')
    plt.figure(figsize=(12, 12))
    plt.title(picture[10:], fontsize=12)
    plt.axis("off") 
    plt.imshow(img)

## Обработка

1 этап обработки - на изображениях оставляем только непосредственно фигурки

In [29]:
min = np.array([90, 1, 0])
max = np.array([120, 255, 255])
edited_images = []
clear_images = []

In [30]:
for image in pictures:
    img = cv.imread(image)
    img_grey = color_change(img, mode='grey')  
    edited_image = canny(img_grey, sigma = 0.6)
    img_hsv = color_change(img, mode='hsv')  
    img_filtered = cv.inRange(img_hsv, lowerb=min, upperb=max)
    edited_image = put_mask(edited_image, img_filtered) #убираем лишние части в картах
    mask = find_the_cards(img)
    edited_image = put_mask(edited_image, mask) #убираем границы карт
    edited_image = dilation(edited_image, disk(1))
    edited_image = fill_holes(edited_image, disk(1))
    edited_image = erosion(edited_image, disk(2))
    edited_images.append(edited_image)

for img in edited_images:
    num = 1
    objects = []
    # перенумеруем все связные компоненты в изображении
    labeled_img = label(img) 
    # проходимся по всем связным компонентам
    for component in regionprops(labeled_img): 
    # убеждаемся в том, что это не шум
        if component.area > 900: 
            # (labeled_img == num) - оставляем в матрице только те пиксели, которые соответствуют фигуре (то есть делаем маску)
            new_object = Object(figure_matrix=(labeled_img == num), centre=component.centroid)
            objects.append(new_object)
        num += 1
    clear_images.append(Image(objects))

2 этап обработки - пытаемся приблизить фигуры многоугольниками, параллельно определяя гладкость, выпуклость и количество вершин

In [31]:
for clear_image in clear_images:
    for figure in clear_image.get_objects():
        flag = True
        contours, _ = cv.findContours(figure.get_figure_matrix().astype(np.uint8), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
        cnt = contours[0]
        # определим гладкость
        eps = 0.01 * cv.arcLength(cnt, True) 
        approx = cv.approxPolyDP(cnt, eps, True) # приближаем нашу фигуру
        figure.set_approx(approx)
        vertexes = figure.get_approx() # получаем уже обработанные вершины
        # определим выпуклость
        if cv.isContourConvex(vertexes):
            figure.set_convex()
        # продолжим определять гладкость
        min = 360
        for i in range(1, len(vertexes)-1):
            vec1 = vertexes[i-1] - vertexes[i]
            vec2 = vertexes[i+1] - vertexes[i]
            angle = angle_between(vec1, vec2)
            if angle < min:
                min = angle
                if not(figure.get_amount_vertexes() >= 10 and min > 98):
                    flag = False
                    break
    
        vec1 = vertexes[-1] - vertexes[0]
        vec2 = vertexes[1] - vertexes[0]
        angle = angle_between(vec1, vec2)
        if angle < min:
            min = angle
            if not(figure.get_amount_vertexes() >= 10 and min > 98):
                flag = False

        vec1 = vertexes[-2] - vertexes[-1]
        vec2 = vertexes[0] - vertexes[-1]
        angle = angle_between(vec1, vec2)
        if angle < min:
            min = angle
            if not(figure.get_amount_vertexes() >= 10 and min > 98):
                flag = False

        if flag:
            figure.set_smoothness()
        else:
            figure.del_smoothness()

        # определим количество вершин
        eps_new = eps*2
        approx_new = cv.approxPolyDP(cnt, eps_new, True) 
        figure.set_approx(approx_new) 

## Результат

In [33]:
for clear_image, image in zip(clear_images, pictures):
    plt.figure(figsize=(12, 12))
    plt.axis("off")
    img = cv.imread(image)
    img = color_change(img, mode='rgb')
    for figure in clear_image.get_objects():
        a = figure.get_approx()
        if figure.get_smoothness() is False:
            cv.drawContours(img, [a], -1, (0,255,0), 3)
            plt.scatter([i[0] for i in a],  [i[1] for i in a], s=20, color='k')
            text = 'P' + str(figure.get_amount_vertexes())
            if figure.get_convex() == False:
                convex = ''
            else:
                convex = 'C'
            text = text + convex
            org = (figure.get_centre()[1]+15, figure.get_centre()[0]-40)
            org = (figure.get_centre()[1], figure.get_centre()[0])
            font = cv.FONT_HERSHEY_SIMPLEX 
            img = cv.putText(img, text, org, cv.FONT_HERSHEY_PLAIN, 2, (251, 255, 0), 2, cv.LINE_AA)
        else:
            cv.drawContours(img, [a], -1, (255, 102, 0), 3)
    img = cv.putText(img, str(clear_image.get_amount()), (10,40), cv.FONT_HERSHEY_PLAIN, 3, (251, 255, 0), 2, cv.LINE_AA)
    plt.imshow(img)
    plt.savefig('output_for_' + image[10:], bbox_inches='tight')